# 03/01/23

Using this file to test the Ranking Set Implementations

In [1]:
import torch, logging, sys
from utils import ranker

import logging
logger = logging.getLogger("lightning")
logger.setLevel("INFO")
logger.addHandler(logging.StreamHandler(sys.stdout))

In [2]:
ranking_set = [
    [0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 1, 1],
    [0, 0, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1],
]

data = torch.tensor(ranking_set, dtype=torch.float)
rs = ranker.RankingSet(store=data, debug=True)

[Ranking Set] Initialized with 4 sample(s)


In [3]:
queries = torch.tensor([[0, 0, 0, 0, 0, 1]], dtype=torch.float)
truth = torch.tensor([[0, 0, 1, 1, 1, 1]], dtype=torch.float)

out = rs.batched_rank(queries, truth)
print(out)

thresh: 
tensor([[0.5000]], device='cuda:0')
ct_greater: 
tensor([[2]], device='cuda:0')
tensor([[0, 0],
        [1, 0]], device='cuda:0')
tensor([[1.0000],
        [0.7071],
        [0.5000],
        [0.4082]], device='cuda:0')
tensor([[2]], device='cuda:0')


03/01/23: Testing a ranking file

In [12]:
ranking_path = "./tempdata/SMILES_ranking_sets/val/rankingset.pt"
rs = ranker.RankingSet(file_path = ranking_path)

[Ranking Set] Initialized with 13756 sample(s)


In [16]:
first = rs.data[0]
first_close = first.clone()
first_close[0] = 1 - first_close[0]

second = rs.data[1]
random_fp = torch.randint(high = 2, size=(6144,), dtype=torch.float32).to(rs.data.device)
random_fp_close = random_fp.clone()
random_fp_close[0] = 1 - random_fp_close[0]

# make sure this random_fp is not in the ranking set, miraculously
assert(torch.all(torch.sum(rs.data * random_fp.view((1, -1)), axis=1) < torch.sum(random_fp)))

# trivial test case: perfect prediction, nonexistent
assert(rs.batched_rank(random_fp.unsqueeze(0), random_fp.unsqueeze(0)) == 0)

# trivial test case: perfect prediction, existent
assert(rs.batched_rank(first.unsqueeze(0), first.unsqueeze(0)) == 0)

# trivial test case: near-prediction, existent
assert(rs.batched_rank(first_close.unsqueeze(0), first.unsqueeze(0)) == 0)

# trivial test case: near-prediction, non-existent
assert(rs.batched_rank(random_fp_close.unsqueeze(0), random_fp.unsqueeze(0)) == 0)